In [4]:
import pandas as pd
import numpy as np

# Multiple linear regression

Classical regression models (linear, logistic) are old and less hype
than recent machine learning models. Nevertheless, given their robustness and
stability in the face of sample fluctuations, their ability to scale up to
massive data... given all these factors mean that they are still widely used
in production especially when the function to be modeled is linear, and it would be
counterproductive to look for something more complicated.


## An introductory example: air ozone prediction

Air pollution, including compounds such as ozone, has become a global concern due to its detrimental effects on human health and the environment. Ozone is a reactive gas formed through complex photochemical reactions involving precursor pollutants such as nitrogen oxides (NOx) and volatile organic compounds (VOCs)3,4,5. Elevated ozone levels in the atmosphere can contribute to respiratory issues, cardiovascular diseases, and lung inflammation in humans. It can also harm plants, reduce crop yields, and disrupt ecosystems. Accurately predicting ozone concentrations in the air is crucial for effective air quality management and the development of appropriate mitigation strategies.

By forecasting ozone levels, policymakers, environmental agencies, and health professionals can take timely measures 
to reduce exposure and mitigate the potential health and ecological risks associated with high ozone concentrations. This can include implementing emission controls, adjusting industrial activities, and raising awareness among vulnerable populations.

Many variables can explain this concentration, such as the wind that pushes air masses. This physical phenomenon is known as advectance (or dilution). Other variables, such as radiation, precipitation, etc., have a definite influence on ozone concentration. We therefore measure other variables likely to have an influence on ozone concentration. Here are a subset of some of these data:

In [5]:
# initialise data of lists.
data = {'O3' :[115.4,76.8,113.8,81.6,115.4,125,83.6,75.2,136.8,102.8],
        'T12':[23.8,16.3,27.2,7.1,25.1,27.5,19.4,19.8,32.2,20.7,],
        'V'  :[9.25, -6.15, -4.92, 11.57, -6.23, 2.76, 10.15, 13.5, 21.27, 13.79],
        'N12':[5, 7, 6, 5, 2, 7, 4, 6, 1, 4]}
df = pd.DataFrame(data)
df.T

,0,1,2,3,4,5,6,7,8,9
O3,115.40,76.80,113.80,81.60,115.40,125.00,83.60,75.2,136.80,102.80
T12,23.80,16.30,27.20,7.10,25.10,27.50,19.40,19.8,32.20,20.70
V,9.25,-6.15,-4.92,11.57,-6.23,2.76,10.15,13.5,21.27,13.79
N12,5.00,7.00,6.00,5.00,2.00,7.00,4.00,6.0,1.00,4.00


The variable $V$ is a synthetic variable. In fact, wind is normally measured in degrees
(direction) and meters per second (speed). The $V$ variable we've created is the wind's projection
of the wind on the East-West axis, so it takes into account both direction and speed.

The $N$ variable is the cloud cover at midday.

To analyze the relationship between temperature $T$ at noon, wind $V$, cloud cover at noon $N$ and ozone $O_3$, we'll look for a function $f$ such that :
$$
O_{3i} \approx f(T_i , V_i , N_i).
$$
minimizing the risk $R_n$ associated with the quadratic loss function
$$
R_n = \sum_{i=1}^n \left(O_{3i} -f(T_i , V_i , N_i)\right)^2.
$$


## The linear model

Minimizing a cost also requires knowledge of the space over which we're minimizing.
the class of functions $\mathcal{F}$ in which we'll assume the true unknown function lies.
The linear model assumes that
$$
\mathcal{F}=\left\{
f : \mathbb{R}^p\rightarrow \mathbb{R};\; f(x^1,\ldots, x^p) = \beta_0 + \sum_{j=1}^p \beta_j x^j
\right\}
$$

The data are assumed to come from the observation of a statistical sample of size
$n$ with $(n > p + 1)$ of $\mathbb{R}^{(p+1)}$.
$$
(x_{1i},\ldots, x^j_i ,\ldots, x^p_i, y_i)\quad i = 1,\ldots, n.
$$

The model is written in matrix form
$$
Y = X \beta + \epsilon
$$
with $Y$ vector of $\mathbb{R}^n$, $X$ matrix $(n,p+1)$, $\beta=(\beta_0,\beta_1,\ldots,\beta_p)^T$ vector
of $\mathbb{R}^{p+1}$ and $\epsilon=(\epsilon_1,\ldots,\epsilon_n)^T$ vector of $\mathbb{R}^n$.

- $Y$ is the variable to be explained
- $X$ is the predictor matrix
- $\epsilon$ is the vector of errors

### Statistical assumptions

1. The $\epsilon_i$ are independent
2. The $X^j$ terms are deterministic
3. Parameters $\beta_0$, $\beta_1$,...,$\beta_p$ are assumed constant
4. The model is Gaussian if $\epsilon_i$ are assumed to have a normal distribution.

### Prepare the data

The data are arranged in a matrix $X$ of size $(n, p+1)$ with general term $X^j$ (column)
and whose first column contains the vector $\mathbb{1}_n$.

Some code
```
import statsmodels.api as sm
X = df[["T12", "V", "N12"]]
X = sm.add_constant(X)
Y = df.O3
```

In [6]:
X = df[["T12", "V", "N12"]]
X

,T12,V,N12
0,23.8,9.25,5
1,16.3,-6.15,7
2,27.2,-4.92,6
3,7.1,11.57,5
4,25.1,-6.23,2
5,27.5,2.76,7
6,19.4,10.15,4
7,19.8,13.50,6
8,32.2,21.27,1
9,20.7,13.79,4


In [7]:
import statsmodels.api as sm

### Least-squares estimation

If we choose to minimize the sum of squared errors, the expression to be minimized in $beta$ is
$$
\sum_{i=1}^n (y_i - \beta_0 - \beta_1 x^1_i - \ldots - \beta_p x^p_i)^2
= \| Y - X\beta \|^2 = Y^T Y - 2\beta^T X^TY+\beta^TX^TX\beta.
$$
The solution to this opitmization problem is
$$
\hat{\beta} = (X^TX)^{-1}X^TY
$$


### Compute the coefficients

Some code
```
B=X.T.dot(Y)
A =X.T.dot(X)
C=pd.DataFrame(np.linalg.inv(A.values), X.columns, X.columns)
beta = C.dot(B)
```

## Results using the OLS function

The Ordinary Least Square are implemented in the library ``statmodels.api``

```
model = sm.OLS(Y, X)
results = model.fit()
results.summary()
```

## Predicting ozone concentration using all variables


We import the data, then use the lm command to regress maxO3 on the other variables in the sample.

In [8]:
import statsmodels.formula.api as smf

In [9]:
ozone = pd.read_csv('ozone.txt', sep=";", decimal=',')

reg_multi = smf.ols('maxO3~T9+T12+T15+Ne9+Ne12+Ne15+maxO3v', data=ozone).fit()
print(reg_multi.summary())

                            OLS Regression Results                            
Dep. Variable:                  maxO3   R-squared:                       0.755
Model:                            OLS   Adj. R-squared:                  0.738
Method:                 Least Squares   F-statistic:                     45.68
Date:                Thu, 14 Mar 2024   Prob (F-statistic):           6.06e-29
Time:                        14:41:00   Log-Likelihood:                -453.71
No. Observations:                 112   AIC:                             923.4
Df Residuals:                     104   BIC:                             945.2
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     12.7055     13.109      0.969      0.3

We can see here that some parameters are not significantly different from 0, because their $p$-values are not less than 5%, the level of testing we want.

The $R^{2}$ is about 0.75, and the adjusted $R^{2}$ is about 0.74.

*This value is higher than in simple linear regression, and this is logical, because when we add potential explanatory variables, we naturally increase the value of these $R^{2}$.*.

## Remove irrelevant variables

Try to get a better model by removing the least significant variable.